In [35]:
import openai
import langchain
import pinecone
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain_community.llms import OpenAI 
from langchain.chains.question_answering import load_qa_chain

In [29]:
from dotenv import load_dotenv
load_dotenv()

False

In [30]:
import os

In [31]:
def read_pdf(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    docs = file_loader.load()
    return docs


In [ ]:
docs = read_pdf('/')
docs

In [ ]:
def chunk_data(docs, chunk_size = 800, chuck_overlap = 50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chuck_overlap)
    doc = text_splitter.split_documents(docs)
    return doc

In [ ]:
documents = chunk_data(docs)
len(documents)

In [ ]:
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

In [ ]:
vectors=embeddings.embed_query("How are you?")
len(vectors)

In [ ]:
## Vector Search DB in Pinecone

## Vector Search DB In Pinecone
pinecone.init(
    api_key="",
    environment="gcp-starter"
)
index_name="langchainvector"

In [ ]:
index=Pinecone.from_documents(documents,embeddings,index_name=index_name)

In [ ]:
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [36]:
llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

C:\Users\Sahil\AppData\Local\Temp\ipykernel_26320\3923786536.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
C:\Users\Sahil\AppData\Local\Temp\ipykernel_26320\3923786536.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_

In [ ]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)